In [ ]:
!nvidia-smi

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [10]:
#import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.python.keras.applications import resnet
from tensorflow.keras.applications import ResNet152V2
tf.keras.applications.resnet_v2.preprocess_input
#from tensorflow.keras.applications.resnet152V2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [11]:
#pip install -U git+https://github.com/keras-team/keras git+https://github.com/keras-team/keras-applications

In [12]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]


In [13]:
train_path = "C:/Users/Vijaya/Tomato_plant/raw/color/train/"
valid_path = "C:/Users/Vijaya/Tomato_plant/raw/color/test/"

In [14]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
import tensorflow
resnet152V2 =tensorflow.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

234553344/234545216 [==============================] - 70s 0us/step


In [15]:
# don't train existing weights
for layer in resnet152V2.layers:
    layer.trainable = False

In [16]:
# useful for getting number of output classes
folders = glob("C:/Users/Vijaya/Tomato_plant/raw/color/train/*")

In [17]:
folders

['C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Bacterial_spot',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Early_blight',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___healthy',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Late_blight',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Leaf_Mold',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Septoria_leaf_spot',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Spider_mites Two-spotted_spider_mite',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Target_Spot',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Tomato_mosaic_virus',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [18]:
# our layers - you can add more if you want
x = Flatten()(resnet152V2.output)

In [19]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet152V2.input, outputs=prediction)

In [20]:
# view the structure of the model
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
 Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)